In [634]:
# %matplotlib inline 
import matplotlib.pyplot as plt
from matplotlib_inline import backend_inline
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.impute import SimpleImputer


data = pd.read_csv(r'C:\Users\Lst12\Downloads\train.csv',index_col=0)
# print(data.head())

# print(data.values[0],'\n', data.columns)
"""分析数据并处理缺失值"""
print(data['Survived'].value_counts(normalize=True))  # 约有62%幸存
print(data['Pclass'].value_counts(normalize=True)) # 1,2,3等舱人数分别占21%,24%,55%
print(data['Sex'].value_counts(normalize=True))   # 女性占65%，男性占35%
print(data['SibSp'].value_counts(normalize=True))    # 没带配偶兄弟姐妹或者只带一两个的占90%以上比例  
print(data['Parch'].value_counts(normalize=True))    # 没带父母子女或者只带一两个的占90%以上比例
print(data['Embarked'].value_counts(normalize=True))  #在S,C,Q三个站登船的占比分别为72.4%,18.9%,8.7%

age_splited = pd.cut(data['Age'],[0, 18, 30, 40, 55, 65, 100])
fare_splited = pd.cut(data['Fare'],[0, 25, 45, 65, 85, 150, 600])

print(fare_splited.value_counts(normalize=True)) # 0~50元费用的占82%
print(age_splited.value_counts(normalize=True)) # 10~40或50岁的乘客占大部分

mean = data['Age'].mean()
data['Age'] = data['Age'].fillna(mean)

mode = data['Embarked'].mode()
data['Embarked'] = data['Embarked'].fillna(mode[0])



Survived
0    0.616162
1    0.383838
Name: proportion, dtype: float64
Pclass
3    0.551066
1    0.242424
2    0.206510
Name: proportion, dtype: float64
Sex
male      0.647587
female    0.352413
Name: proportion, dtype: float64
SibSp
0    0.682379
1    0.234568
2    0.031425
4    0.020202
3    0.017957
8    0.007856
5    0.005612
Name: proportion, dtype: float64
Parch
0    0.760943
1    0.132435
2    0.089787
5    0.005612
3    0.005612
4    0.004489
6    0.001122
Name: proportion, dtype: float64
Embarked
S    0.724409
C    0.188976
Q    0.086614
Name: proportion, dtype: float64
Fare
(0, 25]       0.618721
(25, 45]      0.186073
(45, 65]      0.062785
(65, 85]      0.057078
(85, 150]     0.042237
(150, 600]    0.033105
Name: proportion, dtype: float64
Age
(18, 30]     0.378151
(30, 40]     0.217087
(0, 18]      0.194678
(40, 55]     0.154062
(55, 65]     0.044818
(65, 100]    0.011204
Name: proportion, dtype: float64


In [635]:
"""初步判断各特征重要性"""

survival_about_sex = data.groupby(['Sex'])['Survived'].agg(['mean'])
print(survival_about_sex) #女性存活概率远大于男性，为74%，而男性存活率只有19%左右，说明存活率跟性别有很大关系

survival_about_pclass = data.groupby(['Pclass'])['Survived'].agg(['mean'])
print(survival_about_pclass) # 一等舱存活率最高，为62%,而三等舱仅有24%

survival_about_embarked = data.groupby(['Embarked'])['Survived'].agg(['mean'])
print(survival_about_embarked)  # 在C站上船存活率最高，约为为55%


            mean
Sex             
female  0.742038
male    0.188908
            mean
Pclass          
1       0.629630
2       0.472826
3       0.242363
              mean
Embarked          
C         0.553571
Q         0.389610
S         0.339009


In [636]:
from sklearn.preprocessing import  LabelEncoder,OneHotEncoder
encoder1 =  LabelEncoder()
encoder2 = OneHotEncoder(sparse_output=False)

"""特征工程"""

data['Sex'] = encoder2.fit_transform(data['Sex'].values.reshape(-1,1)).astype(int)
print('原特征名:\n',encoder2.get_feature_names_out())

data['Embarked'] = encoder1.fit_transform(data['Embarked'].values.reshape(-1,1))
print('原特征名:\n',encoder1.classes_)

data['Age_binned'] = encoder1.fit_transform(age_splited.values.reshape(-1,1))
print('原特征名:\n',encoder1.classes_)

data['Fare_binned'] = encoder1.fit_transform(fare_splited.values.reshape(-1,1))
print('原特征名:\n',encoder1.classes_)
print()

data['Family_size'] = data['SibSp'] + data['Parch'] + 1
data['Is_alone'] = (data['Family_size'] == 1).astype(int)

drop_list = ['Name', 'Cabin', 'Ticket', 'Age', 'Fare', 'Family_size', 'SibSp', 'Parch']
data_dropped = data.drop(drop_list, axis=1)
print(data_dropped.head())


原特征名:
 ['x0_female' 'x0_male']
原特征名:
 ['C' 'Q' 'S']
原特征名:
 [Interval(0.0, 18.0, closed='right') Interval(18.0, 30.0, closed='right')
 Interval(30.0, 40.0, closed='right') Interval(40.0, 55.0, closed='right')
 Interval(55.0, 65.0, closed='right')
 Interval(65.0, 100.0, closed='right') nan]
原特征名:
 [Interval(0.0, 25.0, closed='right') Interval(25.0, 45.0, closed='right')
 Interval(45.0, 65.0, closed='right') Interval(65.0, 85.0, closed='right')
 Interval(85.0, 150.0, closed='right')
 Interval(150.0, 600.0, closed='right') nan]

             Survived  Pclass  Sex  Embarked  Age_binned  Fare_binned  \
PassengerId                                                             
1                   0       3    0         2           1            0   
2                   1       1    1         0           2            3   
3                   1       3    1         2           1            0   
4                   1       1    1         2           2            2   
5                   0       3  

d:\Program Files\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Program Files\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Program Files\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [637]:
"""根据特征工程后的特征判断特征重要性"""

survival_about_age = data.groupby(['Age_binned'])['Survived'].agg(['mean'])
print(survival_about_age)  #未成年的儿童存活率最高，年轻人和中年人都差不多，老年人最低

survival_about_fare = data.groupby(['Fare_binned'])['Survived'].agg(['mean'])
print(survival_about_fare)  #总体来说付的越多，存活率越高

survival_about_family = data.groupby(['Is_alone'])['Survived'].agg(['mean'])
print(survival_about_family)   #没有其他家庭成员的存活率低




                mean
Age_binned          
0           0.503597
1           0.355556
2           0.445161
3           0.390909
4           0.343750
5           0.125000
6           0.293785
                 mean
Fare_binned          
0            0.293358
1            0.435583
2            0.581818
3            0.560000
4            0.837838
5            0.689655
6            0.066667
              mean
Is_alone          
0         0.505650
1         0.303538


In [638]:
from sklearn.model_selection import train_test_split, GridSearchCV
X = data_dropped.drop('Survived',axis=1)
y = data_dropped['Survived']
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [639]:
from sklearn.metrics import accuracy_score,classification_report
from sklearn.neighbors import KNeighborsClassifier

"""KNN"""

classifier = KNeighborsClassifier(n_neighbors=3)
classifier.fit(x_train, y_train)
y_pred = classifier.predict(x_test)

print('KNN分类器对测试集的预测值:\n', y_pred)
print('正确值:\n', y_test.values)

accuracy = accuracy_score(y_test, y_pred)
print('预测准确率:',f'{accuracy:.2f}')
print('分类报告:\n', classification_report(y_test, y_pred, target_names=['not_survived','survived']))
# 发现对幸存的召回率特别低，说明真正幸存的只有63%被找到了
# 准确率是做出判断的准确性，召回率是找出真正目标的能力

KNN分类器对测试集的预测值:
 [1 0 0 0 0 0 1 1 0 0 0 0 1 0 1 0 1 0 0 1 1 0 1 1 0 1 1 0 1 0 0 1 0 1 1 1 0
 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 0 1 1 1 1 0 1
 0 0 1 0 1 0 0 1 0 1 0 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 1 0 0 1 1 0 0 1 1 1
 0 1 1 1 0 0 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 1 0 1 0 1 0 1 0 1 0
 0 0 0 1 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
正确值:
 [1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1
 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
预测准确率: 0.75
分类报告:
               precision    recall  f1-score   support

not_survived       0.78      0.82      0.80       110
    survived       0.69      0.64      0.66        69

    accuracy                           0.75       179
   macro avg

In [640]:
from sklearn.linear_model import LogisticRegression

"""Logistic"""

model = LogisticRegression()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print('Logistic模型分类器对测试集的预测:\n', y_pred)
print('真实值:\n', y_test.values)

accuracy = accuracy_score(y_test, y_pred)
print('预测准确率:',f'{accuracy:.2f}')
print('分类报告:\n', classification_report(y_test, y_pred, target_names=['not_survived','survived']))

Logistic模型分类器对测试集的预测:
 [1 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 0 0 1 0 1 0 1 1
 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1 0 1
 0 0 1 0 1 1 0 1 0 1 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1
 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0]
真实值:
 [1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1
 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
预测准确率: 0.76
分类报告:
               precision    recall  f1-score   support

not_survived       0.79      0.84      0.81       110
    survived       0.71      0.64      0.67        69

    accuracy                           0.76       179
   mac

In [641]:
from sklearn.svm import SVC

"""SVM"""

model = SVC()
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print('SVM分类器对测试集的预测:\n', y_pred)
print('真实值:\n', y_test.values)

accuracy = accuracy_score(y_test, y_pred)
print('预测准确率:',f'{accuracy:.2f}')
print('分类报告:\n', classification_report(y_test, y_pred, target_names=['not_survived','survived']))

SVM分类器对测试集的预测:
 [1 1 0 0 0 1 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 0 1 1
 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 0 0 1 0 1 1 0 1 1 0 1
 0 0 0 0 1 1 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1
 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 1 0 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
真实值:
 [1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1
 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
预测准确率: 0.75
分类报告:
               precision    recall  f1-score   support

not_survived       0.78      0.83      0.81       110
    survived       0.70      0.64      0.67        69

    accuracy                           0.75       179
   macro avg 

In [642]:
from sklearn.ensemble import RandomForestClassifier

"""RandomForest"""

model = RandomForestClassifier(criterion='entropy', n_estimators=100, oob_score=True, max_depth=3)
model.fit(x_train, y_train)

y_pred = model.predict(x_test)
print('Randomforest分类器对测试集的预测:\n', y_pred)
print('真实值:\n', y_test.values)

accuracy = accuracy_score(y_test, y_pred)
print('预测准确率:',f'{accuracy:.2f}')
print('分类报告:\n', classification_report(y_test, y_pred, target_names=['not_survived','survived']))

Randomforest分类器对测试集的预测:
 [1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 0 0 1 0 0 0 1 0 1 0
 0 0 1 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 0 0 1 0 0 1 0 0 1 0 1
 0 0 0 0 1 0 0 1 0 0 0 0 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1
 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
真实值:
 [1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1
 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
预测准确率: 0.78
分类报告:
               precision    recall  f1-score   support

not_survived       0.77      0.91      0.83       110
    survived       0.80      0.57      0.66        69

    accuracy                           0.78       179
   m

In [643]:
"""超参数调优"""

param_grid = {
    'criterion':['gini', 'entropy', 'log_loss'],
    'n_estimators': [50, 100, 150, 200],
    'max_depth':[None, 5, 10, 15],
    'min_samples_split':[2, 5, 10, 20],
    'max_features': ['sqrt', 'log2']
}

grid_searcher = GridSearchCV(
    estimator = RandomForestClassifier(random_state=42),  #设置随机种子,保证每轮随机结果都相同，保证模型稳定性
    param_grid=param_grid,
    cv=5,
    scoring='accuracy',
    n_jobs=-1,
) 

grid_searcher.fit(x_train, y_train)
print('最佳参数',grid_searcher.best_params_)
print('最佳交叉验证分数',grid_searcher.best_score_)

最佳参数 {'criterion': 'gini', 'max_depth': 10, 'max_features': 'sqrt', 'min_samples_split': 10, 'n_estimators': 50}
最佳交叉验证分数 0.8160248202501723


In [670]:
best_model = RandomForestClassifier(criterion='gini', max_depth=10, max_features='sqrt', min_samples_split=10, n_estimators=50)

best_model.fit(x_train, y_train)

y_pred = best_model.predict(x_test)
print('Randomforest分类器对测试集的预测:\n', y_pred)
print('真实值:\n', y_test.values)

accuracy = accuracy_score(y_test, y_pred)
print('预测准确率:',f'{accuracy:.2f}')
print('分类报告:\n', classification_report(y_test, y_pred, target_names=['not_survived','survived']))

Randomforest分类器对测试集的预测:
 [1 0 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 0 1 0 1 1 1 0 1 1 1 1 0 0 1 0 1 0 1 0
 0 0 1 0 0 1 1 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 1 1 0 1 1 0 1 0 1
 0 0 1 0 1 0 0 1 0 1 1 1 0 1 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 1 1 1 0 0 1 0 1
 0 1 1 1 0 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 0 1 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0]
真实值:
 [1 1 0 1 0 0 0 1 0 0 0 0 1 1 1 0 1 0 0 1 0 1 0 1 0 1 1 1 1 0 0 1 0 1 0 0 0
 0 0 1 0 0 1 0 1 0 0 1 1 1 1 0 0 0 0 0 1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1 0 1
 0 0 0 0 1 0 0 1 0 0 1 1 0 1 1 0 1 0 1 1 1 0 0 0 0 0 0 1 1 0 1 1 1 0 0 1 1
 0 1 0 1 0 0 0 1 1 0 0 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0
 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 0]
预测准确率: 0.81
分类报告:
               precision    recall  f1-score   support

not_survived       0.82      0.89      0.85       110
    survived       0.80      0.68      0.73        69

    accuracy                           0.81       179
   m

In [671]:

test_data = pd.read_csv(r'C:\Users\Lst12\Downloads\test.csv',index_col=0)

age_splited = pd.cut(test_data['Age'],[0, 18, 30, 40, 55, 65, 100])
fare_splited = pd.cut(test_data['Fare'],[0, 25, 45, 65, 85, 150, 600])

mean = test_data['Age'].mean()
test_data['Age'] = test_data['Age'].fillna(mean)

mode = test_data['Embarked'].mode()
test_data['Embarked'] = test_data['Embarked'].fillna(mode[0])

test_data['Sex'] = encoder2.fit_transform(test_data['Sex'].values.reshape(-1,1)).astype(int)

test_data['Embarked'] = encoder1.fit_transform(test_data['Embarked'].values.reshape(-1,1))

test_data['Age_binned'] = encoder1.fit_transform(age_splited.values.reshape(-1,1))

test_data['Fare_binned'] = encoder1.fit_transform(fare_splited.values.reshape(-1,1))
test_data['Family_size'] = test_data['SibSp'] + test_data['Parch'] + 1
test_data['Is_alone'] = (test_data['Family_size'] == 1).astype(int)

drop_list = ['Name', 'Cabin', 'Ticket', 'Age', 'Fare', 'Family_size', 'SibSp', 'Parch']
test_data_dropped = test_data.drop(drop_list, axis=1)

y_pred = best_model.predict(test_data_dropped)
print('Randomforest分类器对测试集的预测:\n', y_pred)
submission_data = test_data_dropped.copy()
submission_data['Survived'] = y_pred
submission_data.to_csv('Kaggle_Titanic_submission.csv',
                       index=True,
                       columns=['Survived'],
                       header=True,
                       lineterminator='\n',)


Randomforest分类器对测试集的预测:
 [0 0 0 0 0 0 1 0 1 0 0 0 1 0 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1 0 1 0 0
 1 0 0 0 0 0 1 1 0 0 0 1 1 0 0 1 1 0 0 0 0 0 1 0 0 0 1 1 1 1 0 1 1 1 0 1 1
 1 0 0 1 0 1 0 0 0 0 0 0 1 1 1 1 0 0 1 0 1 0 1 0 1 0 1 0 0 0 1 0 0 0 0 0 0
 1 1 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 0 0
 0 0 1 0 0 1 0 0 1 1 0 0 1 0 1 0 0 0 0 0 1 1 0 0 0 0 0 1 1 1 1 1 0 1 1 0 1
 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 1 0 0 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 1 0 0 0 1 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 1 1 1 0 1 0 0 0 0 0 1
 0 0 0 1 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1 0 0 0 0 0 0 0 1 0 0 0 0
 1 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 1 1 0
 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 0 1 1 0 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0
 0 1 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 1 0 0 0 1 0 1 0 0 1 0 1 0 1 0 0
 0 1 0 1 1 1 0 1 0 0 0]


d:\Program Files\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Program Files\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
d:\Program Files\Python312\Lib\site-packages\sklearn\preprocessing\_label.py:120: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
